Loading data

In [13]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [16]:
df = pd.read_csv("Iris.csv", index_col=0)
df.head(2)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa


In [17]:
df['Species'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

Data preparating

In [27]:
df['target'] = df['Species'].map(
    {
        'Iris-setosa':0,
        'Iris-versicolor':1,
        'Iris-virginica':2,
    }
)
df.head(2)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,taregt,target
Id,,,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa,0,0
2,4.9,3.0,1.4,0.2,Iris-setosa,0,0


In [28]:
feature_col = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
target_col = "target"
X = df[feature_col].values
y = df[target_col].values

X.shape, y.shape


((150, 4), (150,))

In [29]:
SEED = 42

In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=SEED, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test, test_size=0.5, random_state=SEED, stratify=y_test
)
X_train.shape, X_val.shape, X_test.shape

((90, 4), (30, 4), (30, 4))

In [31]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)


Data

In [47]:
class DataExtractor(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        X = torch.tensor(self.X[index], dtype=torch.float32)
        y = torch.tensor(self.y[index], dtype=torch.long) #long for int

        return {"X" : X, "y" : y}

In [48]:
train_data_extractor = DataExtractor(X_train, y_train)
valid_data_extractor = DataExtractor(X_val, y_val)
test_data_extractor = DataExtractor(X_test, y_test)

In [49]:
train_data_extractor[2]

{'X': tensor([-0.3126, -0.7730,  0.2290,  0.1268]), 'y': tensor(1)}

In [50]:
train_loader = torch.utils.data.DataLoader(
    train_data_extractor,
    batch_size=8,
    shuffle=True,
)

valid_loader = torch.utils.data.DataLoader(
    valid_data_extractor,
    batch_size=8,
    shuffle=False,
)

test_loader = torch.utils.dsts.DataLoader(
    test_data_extractor,
    batch_size=8,
    shuffle=False,
)